In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import time
import cv2
import os
import imutils
from imutils.video import VideoStream
from matplotlib import pyplot as plt 

In [2]:
haar_data=cv2.CascadeClassifier('facedetection.xml')
#haar_data = cv2.CascadeClassifier(cv2.data.haarcascades + 'facedetection.xml')

from PIL import Image
import matplotlib.image as mpimg

In [3]:
# loading mask detection model which we trained earlier
maskNet = load_model(r"C:\Users\lenovo\ai_proj\models\Model3_da\mobilenet_v2.model")

In [4]:
capture=cv2.VideoCapture(0)
flag,img=capture.read()

while True:
    flag,img=capture.read()
    if flag: 
        #grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        grey = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces =haar_data.detectMultiScale(grey,1.3,5)
        
        for x,y,w,h in faces:            
            face= grey[y:y+w,x:x+w]
            
            #face = cv2.cvtColor(face,cv2.COLOR_GRAY2RGB)   
            #face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            
            face=cv2.resize(face, (224, 224))
            
            face = img_to_array(face)
            face = preprocess_input(face)
            
            face= np.expand_dims(face, axis = 0) # need 4th dim
            
            #face2 = np.array(face, dtype="float32")
            
            #face=face/255.0
            #face=face.reshape(1,224,224,1)
            
            pred =  pred = maskNet.predict(face)[0] # using our trained model
            (mask, withoutMask) = pred
            label = "Mask" if mask > withoutMask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        
            #result=model.predict(face)
            #label=np.argmax(result,axis=1)[0]
            
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            #cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
            cv2.putText(img,label,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2) #(0,0,0)
            
            cv2.imshow('image',img)
            
        if cv2.waitKey(2)==ord("q"):
            break
capture.release()
cv2.destroyAllWindows()